In [4]:
# Importing the required libraries
from dotenv import load_dotenv
from datasets import load_dataset
import os
from openai import OpenAI
from eval_metrics import faithfulness_metric,relevancy_metric
from tqdm import tqdm
from datasets import Dataset, DatasetDict
import json
import pandas as pd

In [5]:
# Specify the path to your .env file
env_path = 'deep_infra.env'

# Load the .env file
load_dotenv(env_path)
api_key = os.getenv('DEEPINFRA_API_TOKEN')

In [ ]:
# --- Initialize OpenAI Client with DeepInfra ---
openai = OpenAI(
    api_key=api_key,
    base_url="https://api.deepinfra.com/v1/openai",
)

# Load dataset
dataset = load_dataset(
    path='BlackFear/dl_project_uncleaned',
    trust_remote_code=True,
)
data = dataset['train']

# Extract data
queries = data['question']
answers = data['answer']
contexts = data['retrieved_contexts']

scores = pd.DataFrame(columns=['id', 'faithfulness', 'relevancy'])
# --- Run Evaluation ---
for i in tqdm(range(len(queries))):
    query = queries[i]
    context = contexts[i]
    answer = answers[i]
    faith = faithfulness_metric(query, context)
    rel = relevancy_metric(query, answer)
    
    scores = scores.append({'id': i, 'faithfulness': faith, 'relevancy': rel}, ignore_index=True)

# Save scores
scores.to_csv('scores.csv', index=False)